In [ ]:
# GAE

In [ ]:
# # Set random seed for reproducibility
# random_seed = 42
# np.random.seed(random_seed)
# 
# # Load datasets
# job_descriptions = pd.read_csv('./data/processed/job_descriptions_processed.csv')
# resumes = pd.read_csv('./data/processed/resume-dataset-processed.csv', converters={'skills': literal_eval})
# 
# # Shuffle job_descriptions and select the first 100 rows
# job_descriptions = job_descriptions.sample(frac=1, random_state=random_seed).head(100)
# 
# # Convert 'skills' column to list
# job_descriptions['skills'] = job_descriptions['skills'].apply(literal_eval)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges, negative_sampling
import pandas as pd
import uuid
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, average_precision_score

from ast import literal_eval
import numpy as np

from torch_geometric.nn import Node2Vec
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Load datasets
job_descriptions = pd.read_csv('./data/processed/job_descriptions_processed.csv')
resumes = pd.read_csv('./data/processed/general-resume-dataset-processed-v1.csv', converters={'skills': literal_eval})

# Shuffle job_descriptions and select the first N rows
job_descriptions = job_descriptions.sample(frac=1, random_state=random_seed).head(10000)

# Convert 'skills' column to list
job_descriptions['skills'] = job_descriptions['skills'].apply(literal_eval)

In [ ]:
# Replace None values in job_title and category with a default value before encoding
job_descriptions['job_title'].fillna('unknown', inplace=True)
resumes['job_title'].fillna('unknown', inplace=True)
resumes['category'].fillna('unknown', inplace=True)

# Ensure job_ids and candidate_ids are correctly assigned
job_descriptions['job_id'] = range(1, len(job_descriptions) + 1)
resumes['candidate_id'] = range(1, len(resumes) + 1)

# Add 'unknown' to the list of all titles and categories to handle unseen labels
all_titles = job_descriptions['job_title'].tolist() + resumes['job_title'].tolist()
all_titles.append('unknown')
all_categories = resumes['category'].tolist()
all_categories.append('unknown')

In [ ]:
# Fit the label encoders
le_job_title = LabelEncoder()
le_category = LabelEncoder()
le_job_title.fit(all_titles)
le_category.fit(all_categories)

# Transform the columns
job_descriptions['job_title'] = le_job_title.transform(job_descriptions['job_title'])
resumes['job_title'] = le_job_title.transform(resumes['job_title'])
resumes['category'] = le_category.transform(resumes['category'])

# Encode skills
all_skills = set(skill for skills in job_descriptions['skills'].tolist() + resumes['skills'].tolist() for skill in skills)
le_skills = {skill: i for i, skill in enumerate(all_skills)}

In [ ]:
# Create nodes and edges for the graph
nodes = []
edges = []
node_features = []

# Add job nodes
for i, row in job_descriptions.iterrows():
    nodes.append(row['job_id'])
    skills_vector = [0] * len(le_skills)
    if row['skills']:  # Check if skills are not empty
        for skill in row['skills']:
            skills_vector[le_skills[skill]] = 1
    node_features.append([row['job_title']] + skills_vector)

# Add resume nodes, using 'category' instead of 'job_title'
for i, row in resumes.iterrows():
    nodes.append(row['candidate_id'] + len(job_descriptions))
    skills_vector = [0] * len(le_skills)
    if row['skills']:  # Check if skills are not empty
        for skill in row['skills']:
            skills_vector[le_skills[skill]] = 1
    node_features.append([row['job_title']] + skills_vector)

# Add edges based on job_title overlap
# for i, job in job_descriptions.iterrows():
#     for j, resume in resumes.iterrows():
#         if job['job_title'] == resume['job_title']:  # Changed from resume['job_title'] to resume['category']
#             edges.append((job['job_id'], resume['candidate_id'] + len(job_descriptions)))

def skills_overlap(job_skills, resume_skills):
    return len(set(job_skills).intersection(set(resume_skills)))

threshold = 1  # Set a threshold for the minimum number of overlapping skills

# for i, job in job_descriptions.iterrows():
#     for j, resume in resumes.iterrows():
#         overlap = skills_overlap(job['skills'], resume['skills'])
#         if overlap >= threshold:
#             edges.append((job['job_id'], resume['candidate_id'] + len(job_descriptions)))

for i, job in job_descriptions.iterrows():
    for j, resume in resumes.iterrows():
        if job['job_title'] == resume['job_title'] or skills_overlap(job['skills'], resume['skills']) >= threshold:
            edges.append((job['job_id'], resume['candidate_id'] + len(job_descriptions)))

In [ ]:
# Correct node indices
num_job_nodes = len(job_descriptions)
num_resume_nodes = len(resumes)
total_nodes = num_job_nodes + num_resume_nodes

# Reindex edges to ensure correct range
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
edge_index[0] = edge_index[0].clamp(0, total_nodes - 1)
edge_index[1] = edge_index[1].clamp(0, total_nodes - 1)

# Convert to torch tensors
x = torch.tensor(node_features, dtype=torch.float)

# Create PyTorch Geometric data object
data = Data(x=x, edge_index=edge_index)

# Ensure edge indices are within range
data.edge_index = edge_index
data.num_nodes = total_nodes
print(data.edge_index)
print(data.num_nodes)

# Splitting edges for training/validation
original_edge_index = data.edge_index.clone()
original_pos_edge_index = data.edge_index.clone()
original_neg_edge_index = data.edge_index.clone()
data = train_test_split_edges(data)

# Manually create negative edges for training
neg_edge_index_train = negative_sampling(
    edge_index=data.train_pos_edge_index,
    num_nodes=data.num_nodes,
    num_neg_samples=data.train_pos_edge_index.size(1)
)
data.train_neg_edge_index = neg_edge_index_train

# Manually create negative edges for testing
neg_edge_index_test = negative_sampling(
    edge_index=data.test_pos_edge_index,
    num_nodes=data.num_nodes,
    num_neg_samples=data.test_pos_edge_index.size(1)
)
data.test_neg_edge_index = neg_edge_index_test

In [137]:
# Define number of nodes
num_nodes = data.num_nodes

# Node2Vec parameters
embedding_dim = 64
walk_length = 20
context_size = 10
walks_per_node = 10
batch_size = 64
lr = 0.1
num_epochs = 100

In [138]:
# Initialize Node2Vec
node2vec = Node2Vec(
    edge_index=original_edge_index,
    embedding_dim=embedding_dim,
    walk_length=walk_length,
    context_size=context_size,
    walks_per_node=walks_per_node,
    num_negative_samples=1,
    p=1,
    q=1,
    sparse=True,
    num_nodes=num_nodes  # Specify number of nodes
)

# Move to the appropriate device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
node2vec = node2vec.to(device)

In [139]:
# Optimizer for Node2Vec
optimizer = torch.optim.SparseAdam(node2vec.parameters(), lr=lr)

In [140]:
# Training loop for Node2Vec
def train_node2vec(num_epochs):
    node2vec.train()
    for epoch in range(num_epochs):
        total_loss = 0
        loader = node2vec.loader(batch_size=batch_size, shuffle=True, num_workers=0)  # Set num_workers=0
        for i, (pos_rw, neg_rw) in enumerate(loader):
            optimizer.zero_grad()
            loss = node2vec.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if i % 10 == 0:
                print(f'Epoch {epoch + 1}, Iteration {i}, Loss: {total_loss / 10}')
                total_loss = 0

In [141]:
# Train Node2Vec model
train_node2vec(num_epochs)

Epoch 1, Iteration 0, Loss: 0.3731998443603516
Epoch 1, Iteration 10, Loss: 3.2722758769989015
Epoch 1, Iteration 20, Loss: 2.6165997266769407
Epoch 1, Iteration 30, Loss: 2.043922543525696
Epoch 1, Iteration 40, Loss: 1.6112670063972474
Epoch 1, Iteration 50, Loss: 1.29803067445755
Epoch 1, Iteration 60, Loss: 1.0976505279541016
Epoch 1, Iteration 70, Loss: 0.9791308343410492
Epoch 1, Iteration 80, Loss: 0.9161386787891388
Epoch 1, Iteration 90, Loss: 0.8717039287090301
Epoch 1, Iteration 100, Loss: 0.8507448554039001
Epoch 1, Iteration 110, Loss: 0.839344573020935
Epoch 1, Iteration 120, Loss: 0.8344492435455322
Epoch 1, Iteration 130, Loss: 0.8386867642402649
Epoch 1, Iteration 140, Loss: 0.8440695941448212
Epoch 1, Iteration 150, Loss: 0.8507992148399353
Epoch 1, Iteration 160, Loss: 0.8593101918697357
Epoch 1, Iteration 170, Loss: 0.869745534658432
Epoch 1, Iteration 180, Loss: 0.8807528734207153
Epoch 1, Iteration 190, Loss: 0.8956387758255004
Epoch 2, Iteration 0, Loss: 0.088761

In [142]:
# Extract embeddings
node_embeddings = node2vec.embedding.weight.data.cpu().numpy()

# Update node features with embeddings
data.x = torch.tensor(node_embeddings, dtype=torch.float)

In [143]:
class GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode(self, z, pos_edge_index, neg_edge_index):
        pos_pred = (z[pos_edge_index[0].long()] * z[pos_edge_index[1].long()]).sum(dim=1)
        neg_pred = (z[neg_edge_index[0].long()] * z[neg_edge_index[1].long()]).sum(dim=1)
        return pos_pred, neg_pred

    def forward(self, data):
        z = self.encode(data.x, data.train_pos_edge_index)
        return z

In [144]:
# Initialize and train GAE model as before
model = GAE(data.num_node_features, 32)  # Adjust dimensions as needed
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [145]:
def train(data):
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.train_pos_edge_index)
    pos_pred, neg_pred = model.decode(z, data.train_pos_edge_index, data.train_neg_edge_index)
    pos_loss = loss_fn(pos_pred, torch.ones_like(pos_pred))
    neg_loss = loss_fn(neg_pred, torch.zeros_like(neg_pred))
    loss = pos_loss + neg_loss
    if torch.isnan(loss) or torch.isinf(loss):
        print("Warning: NaN or Inf loss detected")
        return float('inf')
    loss.backward()
    optimizer.step()
    return loss.item()

In [146]:
for epoch in range(1000):
    loss = train(data)
    if loss == float('inf'):
        break
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

Epoch 0, Loss: 1.5716328620910645
Epoch 100, Loss: 0.9737919569015503
Epoch 200, Loss: 0.934123158454895
Epoch 300, Loss: 0.9196598529815674
Epoch 400, Loss: 0.9115588665008545
Epoch 500, Loss: 0.9057427644729614
Epoch 600, Loss: 0.901170015335083
Epoch 700, Loss: 0.8973367810249329
Epoch 800, Loss: 0.8938947319984436
Epoch 900, Loss: 0.8904709219932556


In [147]:
def evaluate_model(data, model):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.train_pos_edge_index)
        pos_pred = torch.sigmoid((z[data.test_pos_edge_index[0].long()] * z[data.test_pos_edge_index[1].long()]).sum(dim=1)).cpu().numpy()
        neg_pred = torch.sigmoid((z[data.test_neg_edge_index[0].long()] * z[data.test_neg_edge_index[1].long()]).sum(dim=1)).cpu().numpy()

    y_true = np.concatenate([np.ones(pos_pred.shape[0]), np.zeros(neg_pred.shape[0])])
    y_pred = np.concatenate([pos_pred, neg_pred])

    auc_roc = roc_auc_score(y_true, y_pred)
    ap = average_precision_score(y_true, y_pred)

    return auc_roc, ap

In [148]:
# Example usage
auc_roc, ap = evaluate_model(data, model)
print(f"AUC-ROC: {auc_roc:.4f}, AP: {ap:.4f}")

AUC-ROC: 0.9703, AP: 0.9558


In [149]:
def predict_best_candidates(job_descriptions, resumes, z):
    job_ids = job_descriptions['job_id'].values
    candidate_ids = resumes['candidate_id'].values + len(job_descriptions)

    predictions = []
    for job_id, job_title, job_skills in zip(job_ids, job_descriptions['job_title'], job_descriptions['skills']):
        best_match = None
        best_score = float('-inf')
        for candidate_id, category, candidate_job_title, skills in zip(
                resumes['candidate_id'], resumes['category'], resumes['job_title'], resumes['skills']):
            candidate_index = candidate_id + len(job_descriptions) - 1
            job_index = job_id - 1

            # Ensure indices are within range
            if candidate_index >= z.size(0) or job_index >= z.size(0):
                continue

            score = torch.sigmoid(torch.dot(z[job_index], z[candidate_index])).item()
            if score > best_score:
                best_score = score
                best_match = (job_id, job_title, candidate_id, candidate_job_title, category, skills, score)

        if best_match:
            job_id, job_title, candidate_id, candidate_job_title, category, skills, score = best_match
            match_percentage = score * 100  # Assuming the score is between 0 and 1
            predictions.append({
                "Job ID": job_id,
                "Job Title": le_job_title.inverse_transform([job_title])[0],
                "Candidate ID": candidate_id,
                "Candidate Job Title": le_job_title.inverse_transform([candidate_job_title])[0],
                "Candidate Category": le_category.inverse_transform([category])[0],
                "Match Percentage": match_percentage,
                "Mutual Skills": set(job_skills).intersection(set(skills)),
                "Job Skills": job_skills,
                "Candidate Skills": skills
            })

    predictions_df = pd.DataFrame(predictions)
    return predictions_df

In [150]:
# Example usage
with torch.no_grad():
    z = model.encode(data.x, data.train_pos_edge_index)

In [151]:
predictions_df = predict_best_candidates(job_descriptions.sample(frac=1, random_state=random_seed)[:100], resumes, z)

torch.save(model.state_dict(), f"./models/gcn-recommendation-system-{auc_roc:.2f}-acc-{uuid.uuid4()}-{time.strftime('%Y%m%d-%H%M%S')}-v1.pth")

In [152]:
predictions_df.head(1000)

,Job ID,Job Title,Candidate ID,Candidate Job Title,Candidate Category,Match Percentage,Mutual Skills,Job Skills,Candidate Skills
0,6253,network administrator,1646,product specialists,apparel,93.553579,{},"[system administration, server maintenance, ac...","[business, support, front end, documentation, ..."
1,4685,customer service representative,1022,sales,sales,93.024325,{},"[live chat support, online customer communicat...","[business, support, medium, commerce, bash, ma..."
2,1732,procurement manager,1632,jobs coordinator,apparel,99.999988,{},"[procurement processes, vendor assessment, con...","[business, release, business administration, s..."
3,4743,account executive,1022,sales,sales,86.895841,{},"[key account management, sales strategy develo...","[business, support, medium, commerce, bash, ma..."
4,4522,legal counsel,2236,principal consultant,banking,98.519576,{},"[intellectual property law, trademark and pate...","[interaction, business, project management, wo..."
...,...,...,...,...,...,...,...,...,...
95,3788,supply chain manager,1968,construction safety officer,construction,99.876010,{},"[demand forecasting, inventory management, dat...",[monitoring]
96,9190,ux/ui designer,800,group fitness instructor,fitness,82.718599,{},"[u, i design principles and best practices, gr...",[]
97,7826,graphic designer,1646,product specialists,apparel,90.890688,{},"[user interface, u, i, user experience, u, x, ...","[business, support, front end, documentation, ..."
98,7540,procurement specialist,1604,stylist,apparel,64.016950,{},"[procurement strategies, supplier management, ...","[support, business, accounting, marketing]"
